In [1]:
import base64
from google import genai
from google.genai import types
import base64


def load_m4a_as_bytes(filepath):
  """
  Loads a local m4a file as bytes, mimicking the base64 decoded data.

  Args:
    filepath: The path to the m4a file.

  Returns:
    bytes: The content of the m4a file as bytes.  Returns None if the file
           cannot be opened.
  """
  try:
    with open(filepath, 'rb') as f:  # Open in binary read mode
      audio_bytes = f.read()
      return audio_bytes
  except FileNotFoundError:
    print(f"Error: File not found at {filepath}")
    return None
  except Exception as e:
    print(f"Error reading file: {e}")
    return None
  

def read_prompt(filename: str) -> str:
    """Load system prompt from file."""
    with open(filename, 'r') as f:
        return f.read()

In [9]:
def generate(audio_bytes):
  client = genai.Client(
      vertexai=True,
      project="rd-ri-genai-dev-2352",
      location="us-central1",
  )
  audio1 = types.Part.from_bytes(
      data=audio_bytes,
      mime_type="audio/x-m4a",
  )

  model = "gemini-2.0-pro-exp-02-05"
  #model = "gemini-2.0-flash-001"
  contents = [
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text=read_prompt("./prompt.md")),
        audio1
      ]
    )
  ]
  generate_content_config = types.GenerateContentConfig(
    temperature = 0.5,
    top_p = 0.95,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
  )
  response = client.models.generate_content(
    model = model,
    contents = contents,
    config = generate_content_config)
  return response



In [10]:
# Example usage:
file_path = "./Interview Ben.m4a"  # Replace with the actual path to your m4a file
audio_data = load_m4a_as_bytes(file_path)

response = generate(audio_data)
#print(response.text)

In [5]:
print(response.text)

I: So, ich hoffe man hat mich verstanden. ein bisschen Steinzeug, glaube ich. Genau. ja, hallo, vielen Dank für deine Teilnahme im Rahmen meiner Masterarbeit möchte ich dich interviewen. Das Gespräch wird aufgezeichnet und wird einem transkribiert. Deine Daten werden anonymisiert und vertraulich behandelt und nur für Forschungszwecke verwendet. Bist du damit einverstanden?
R: Ja.
I: Okay, starten wir mit deinem Alter. Wie alt bist du?
R: Ich bin 27 Jahre alt.
I: Was ist dein höchster Bildungsgrad oder aktuelle Ausbildung?
R: Ich bin Bachelor of Arts und bin gerade in Ausbildung zum Diplomingenieur Architekten.
I: Okay. Welche Sprachen sprichst du?
R: Ich spreche fließend Deutsch und Englisch und Basic Spanisch und Französisch.
I: Okay, super. hier brauchen wir jetzt Deutsch und Englisch, das ist schon mal gut. Du kannst aber auch jederzeit, wie gesagt, das Übersetzungstool nutzen für einzelne Bereiche. Genau. Vielen Dank, dass du teilnimmst. In dieser Studie untersuchen wir einen neuen

In [7]:
# Save to file
with open("output/transcription.md", "w", encoding="utf-8") as f:
      f.write(response.text)